In [43]:
question_text = "What is the aim of experiment?"
text = [
  "After a 16-year hiatus, Russia resumed in 2013 its program of biomedical research in space, with the successful 30-day flight of the Bion-M 1 biosatellite (April 19–May 19, 2013), a specially designed automated spacecraft dedicated to life-science experiments. “M” in the mission's name stands for “modernized”; the epithet was equally applicable to the spacecraft and the research program. The principal animal species for physiological studies in this mission was the mouse (Mus musculus). Unlike more recent space experiments that used female mice, males were flown in the Bion-M 1 mission. The challenging task of supporting mice in space for this unmanned, automated, 30-day-long mission, was made even more so by the requirement to house the males in groups.",

  "Russian biomedical research in space traditionally has employed dogs, rats, monkeys, and more recently Mongolian gerbils. The flight of Laika in 1957 was one of the early dog experiments and became world famous for demonstrating that a living organism can withstand rocket launch and weightlessness, thus paving the way for the first human spaceflight. Laika's success also promoted biomedical research with other non-human animals in space that culminated with the Bion biosatellites program. A total of 212 rats and 12 monkeys were launched on 11 satellites and exposed in microgravity for 5.0–22.5 days between 1973 and 1997. Animal experiments on the Bion missions have contributed comprehensive data on adaptive responses of sensorimotor, cardiovascular, muscle, bone and other systems to spaceflight conditions and the mechanisms underlying these adaptations [1], [2].",

  "The use of mice for space experiments offers numerous advantages. Probably the most apparent one is their small size and thus the possibility of utilizing more animals per flight, thus increasing scientific output and the cost-efficiency ratio. Comparisons of data obtained with mice, with those obtained from larger species or humans can also reveal how factors affecting adaptation to spaceflight conditions depend on the size of the organism. The mouse has become the most prevalent “mammalian model” in biomedical research, with a fully described genome and an established role in genetically engineered mutants. While mice are preferred mammalian models for molecular biology studies, their small size is a debated limitation rather than an advantage for physiological studies. Miniaturization of scientific hardware has reduced some of the disadvantages of the species small size. Finally, the use of genetically controlled mice offers a means to reduce inter-individual variability and obtain potentially more consistent results.",

  "Despite the advantages of the mouse as a model organism for space research, their use was rather limited (apart from a number of experiments with mice during early space exploration in the 1950's and 1960's, which were aimed primarily at testing if living organisms can survive the launch or a brief exposure in microgravity) [3]. Flight experiments with mice were performed aboard STS-90 (“NeuroLab”), STS-108, STS-129, STS-131 (“Mouse Immunology I”), STS-133 (“Mouse Immunology II”), and STS-135 with exposure times ranging from 12 to 16 days. Research programs of these flights were largely focused on studies of muscle, bone/tendon/cartilage, nervous, and cardiovascular systems, and innate and acquired immune responses. Experiments were performed with groups of 30 or fewer female C57BL/6J mice, which were dissected typically shortly after return [4]–[9].",

  "The Mice Drawer System (MDS) experiment of the Italian Space Agency is by far the longest spaceflight of mice to date [10]. In this mission, 6 mice were exposed for 91 days aboard the International Space Station. The advantages offered by the possibility of genetic manipulations with mice were utilized in this experiment; three mice were transgenic with pleiotrophin overexpression (C57BLJ10/ PTN) and three mice were their wild-type counterparts. The MDS habitats required periodic replenishment and servicing by by astronauts. Sadly, half of the mice died during the course of this mission due to various estimated reasons.",

  "In the present paper we aim to present a brief overview of the Bion-M 1 mission scientific goals and experimental design. Of particular interest we will focus on the program of mouse training and selection for the experiments, and some outcomes of the Bion-M 1 mission."]


model_name = "meta-llama/Llama-3.1-8B-Instruct"
style = "fun"
level = "student"
max_token = 20
temperature = 0.1
task = "text-generation"

embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

template_text = """
              System : You are a biology rag chatbot, you will be given a context and a question from that context and you have to answer that question with {style}, and considering him as {level}
              User :  Explain the following query -
                      {query} with {style} and assuming user to be {level}
              Context regarding query: {required_text_for_query}

              Only give answer the question when you are sure of it and it is present in Context somewhere.
              Also consider the max token limit to be {max_token},do not go over this token limit.
    """
template_variables=['query','style','level','max_token','required_text_for_query']



In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN
print("HF token set?", bool(os.environ.get("HUGGINGFACEHUB_API_TOKEN")))

HF token set? True


In [45]:
%pip install -q langchain-huggingface transformers accelerate text-generation
%pip install sentence-transformers

In [46]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace , HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.prompts import PromptTemplate
import numpy as np

In [47]:
embeddings = HuggingFaceEmbeddings(model_name = embedding_model)
vector = embeddings.embed_documents(text)
question_vector = embeddings.embed_query(question_text)

In [48]:
score = cosine_similarity([question_vector],vector)[0]
sorted_score = sorted(enumerate(score),key = lambda x:x[1])
top_score_index = sorted_score[0][0]
second_top_score_index = sorted_score[1][0]
required_text_for_query =  text[top_score_index] + "\n\n" + text[second_top_score_index]



In [49]:
print(required_text_for_query)

The Mice Drawer System (MDS) experiment of the Italian Space Agency is by far the longest spaceflight of mice to date [10]. In this mission, 6 mice were exposed for 91 days aboard the International Space Station. The advantages offered by the possibility of genetic manipulations with mice were utilized in this experiment; three mice were transgenic with pleiotrophin overexpression (C57BLJ10/ PTN) and three mice were their wild-type counterparts. The MDS habitats required periodic replenishment and servicing by by astronauts. Sadly, half of the mice died during the course of this mission due to various estimated reasons.

Despite the advantages of the mouse as a model organism for space research, their use was rather limited (apart from a number of experiments with mice during early space exploration in the 1950's and 1960's, which were aimed primarily at testing if living organisms can survive the launch or a brief exposure in microgravity) [3]. Flight experiments with mice were perfor

In [50]:
prompt = PromptTemplate(
    template = template_text,
    input_variables = template_variables
)

In [51]:

llm = HuggingFaceEndpoint(
    repo_id = model_name,
    task = task,
    temperature = temperature,

)
model = ChatHuggingFace(llm=llm)

In [52]:
template = prompt.invoke({
    'query': question_text,
    'style' : style,
    'level' : level,
    'max_token': max_token,
    'required_text_for_query' : required_text_for_query,}
    )

In [53]:
result = model.invoke(template)
print(result.content)

Let's talk about the Mice Drawer System experiment!

So, you want to know... **What is the aim of an experiment?**

Well, my curious student, the aim of an experiment is to **TEST A HYPOTHESIS** or to **ANSWER A QUESTION**! In the case of the MDS experiment, the aim was to study how mice behave and respond in space, and to learn more about the effects of microgravity on their bodies.

Think of it like a science adventure, where you're trying to solve a mystery or answer a question, and the experiment is your way of gathering clues and evidence to figure it out!
